In [1]:
import pandas as pd
from pandarallel import pandarallel
pandarallel.initialize(shm_size_mb=2,nb_workers=90,progress_bar=False,verbose=1)
import numpy as np
import matplotlib.pyplot as plt
import pickle

import gensim
from gensim import corpora, models
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
from gensim.models import CoherenceModel

from pprint import pprint

import pyLDAvis
import pyLDAvis.gensim_models

def write_pickle(obj_in,out_path_in,name_in):
    pickle.dump(obj_in,open(out_path_in+name_in+".pk","wb"))
    
def read_pickle(path_in, file_name):
    tmp=pickle.load(open(path_in + file_name + ".pk", "rb"))
    return tmp
out_path="C:/Users/dadia/Desktop/FastAI/"

import warnings
warnings.simplefilter('ignore')

ModuleNotFoundError: No module named 'gensim'

In [66]:
seven=pd.read_csv("/mnt/vaccine/old_dataset/topic_of_tweets/final_final_sorted_texts/7.csv")
ten=pd.read_csv("/mnt/vaccine/old_dataset/topic_of_tweets/final_final_sorted_texts/10.csv")
fourteen=pd.read_csv("/mnt/vaccine/old_dataset/topic_of_tweets/final_final_sorted_texts/14.csv")

In [67]:
text=pd.DataFrame(seven["cleaned_text"]+ten["cleaned_text"]+fourteen["cleaned_text"])

In [68]:
text=text.drop_duplicates(keep='first', inplace=False)
text=text.dropna()

In [69]:
text

cleaned_text
0    true don't put students and parents at risk po...
1    learning to live with covid and myself lifesty...
2    no trial all trials over top secret data it wi...
3    telling kevin about enrolling in the trial tod...
4    i now desperately want to buy again those wale...
..                                                 ...
395  if caught using a fake card to get into places...
396  i see a few people outgassing about csl share ...
397  look as a juris doctor i can tell you that liv...
398  catholics the pope has said getting vaccinated...
399  in vaccine science a reduced immune response i...

[400 rows x 1 columns]

In [70]:
def extract_senti(var_in):
    tmp = "0"
    try:
        from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
        senti_t = SentimentIntensityAnalyzer()
        tmp = senti_t.polarity_scores(var_in)
        tmp = tmp["compound"]
    except:
        pass
    return tmp

In [71]:
text["senti"]=text.cleaned_text.parallel_apply(extract_senti)

In [72]:
def preprocess(text):
    import gensim
    import nltk
    from gensim.utils import simple_preprocess
    from gensim.parsing.preprocessing import STOPWORDS
    from nltk.stem import WordNetLemmatizer, SnowballStemmer
    from nltk.stem.porter import PorterStemmer
    import numpy as np
    ps=PorterStemmer()
    np.random.seed(2018)
    result=[]
    sw=set(gensim.parsing.preprocessing.STOPWORDS).union(set(["vaccine","virus","covid","covid-19","vaccination","corona","coronavirus","unvaccinated","vaccinated","vaccines","people","dose","shot","doses"]))
    try:
        for token in gensim.utils.simple_preprocess(text):
            if token not in sw and len(token)>3:
                result.append(WordNetLemmatizer().lemmatize(ps.stem(token),pos='v'))
    except:
        print(text)
        pass
    return result

In [73]:
processed_doc=text.cleaned_text.parallel_apply(preprocess)

In [74]:
processed_doc

0      [true, student, parent, risk, postpon, exam, g...
1      [learn, live, lifestyl, tip, life, style, guid...
2      [trial, trial, secret, data, market, boss, put...
3      [tell, kevin, enrol, trial, today, swab, wors,...
4      [desper, want, wale, popcorn, worth, curs, own...
                             ...                        
395    [catch, fake, card, place, requir, receiv, hef...
396    [outgass, share, price, bother, actual, look, ...
397    [look, juri, doctor, tell, livestock, prevent,...
398    [cathol, pope, say, get, moral, oblig, piss, p...
399    [scienc, reduc, immun, respons, elderli, know,...
Name: cleaned_text, Length: 400, dtype: object

In [75]:
tweets_dictionary=gensim.corpora.Dictionary(processed_doc)
bow_corpus=[tweets_dictionary.doc2bow(doc) for doc in processed_doc]
tfidf=models.TfidfModel(bow_corpus)
tfidf_corpus=tfidf[bow_corpus]

In [76]:
trust=list(['I dont trust the government. The whole thing is a conspiracy.Those companies just want to make money', 
            'Dont listen to these companies. Their testing data is fake.',
            'Government doesnt understand covid. Vaccine not gonna help. Its gonna make it worse.',
            'The pharmaceuticals are exploiting american people',
            'I dont trust pfizer vaccine',
            "Ivermectin was approved for COVID-19 treatment, which proved the correctness of conspiracy theory again.",
            "Now the truth is that there is no epidemic at all, the epidemic prevention work is just a fig leaf for clearing population.",
            "Human beings have not developed a vaccine that can effectively control for 82 years, but have produced many kinds of covid-19 vaccines within one year. There are only two possibilities: either novel coronavirus is fake or covid-19 vaccine is fake",
            "I heard that in some places, the subsidy for vaccination has reached as high as 1000 dollars. I don't think it's worth so much money. It's a human experiment with our bodies",
            "Does the government dare to disclose the composition of the vaccine? Does the government dare to count the data of adverse reactions and mortality after vaccination"])

safety=list(['I have a headache. Its the covid vaccine.',
            'All my family are experiencing side effects of the vaccine',
            'The companies sell these vaccine before long term test. Safety is not ensured.',
            'Vaccine hurts children and pregnant women especially',
            "After more than a year, I was tortured by the covid-19 vaccine again. Last year, after receiving the first injection, I almost died. I had difficulty breathing all day, had short breath, and had low blood pressure.",
            "I don't recommend vaccination for children. No one knows what genetic side effects there will be in the future, whether there will be DNA mutations or not.",
            "There is no clinical trial data can be found on the Internet. All we can find are side effects.",
            "Does anyone get urticaria after taking the vaccine? I decide not to take the third injection anymore.",
            "I got dermatitis after the second shot of vaccine. I took a lot of medicine to cure it. I didn't dare to have the third injection.",
            "After I received the covid-19 vaccine at the end of last year, my menstrual period was not normal anymore. It was impossible to get a third injection of the vaccine"])

efficacy=list(['I caught covid after taking vaccine. Vaccines dont work at all.',
              'mRNA vaccines dont work. mRNA is a false science. ',
              'russian vaccines dont do anything to the virus. people still get virus after vaccine',
              'pfizer vaccine dont stop the virus.Vaccine does not imporve immunity',
              'vaccines cannot deal with mutated virus. Covid mutates really fast. My friends get the new strand of covid right after getting the vaccine',
              "I don't think the vaccine can prevent the virus, so it's better not to vaccinate.",
              "The covid-19 is just like the flu. Vaccination does not affect whether you will be infected",
              "It is a proven fact that vaccination cannot avoid infection. Many people were infected with covid-19 just a few days after receiving their third dose of vaccine",
              "is the vaccine really effective?",
              "vaccines is not a way to end covid. vaccines dont function for virus"])

access=list(['I have to drive for 4 hours to get vaccinated.',
            'The queues for vaccine is always too long. Need more medical workers',
            'medical centre closed for weeks',
            'not enough vaccines delivered to our county. I have waited for weeks',
            'no place to get vaccinated.please help',
            "I haven't been vaccinated at all, mainly because I'm lazy.",
            "The vaccination place is too far away. I don't want to waste so much time",
            "Once I went to the vaccination place and found that there was a long line, so I didn't want to get vaccinated",
            "There is no vaccination site near my home. If I want to get vaccinated, I have to go to a far hospital. I don't think it is worth spending my so much time",
            "I went to a vaccination site once. When I wanted to get vaccinated, I found that the vaccination process was too complicated. It required not only an ID card but also a mobile phone to scan the code, so I didn't get vaccinated due to the complex processes"])

ct=list([trust,safety,efficacy,access])

In [77]:
def form_vector(lda_top_num,as_tweet,lda):
    as_top_num=len(as_tweet)
    tweet_num=len(as_tweet[0])
    vectors=np.zeros([as_top_num,tweet_num,lda_top_num])  #(assigned topics numbers,number of tweets generated in each assigned topic,number of lda topics)
    for topic_ind in range(0,as_top_num):
        for tweet_ind in range(0,tweet_num):
            unseen_document=as_tweet[topic_ind][tweet_ind]
            bow_vector=tweets_dictionary.doc2bow(preprocess(unseen_document))
            #print(lda[bow_vector])
            top_dist=dict(lda.get_document_topics(bow_vector,minimum_probability=0))
            for k in list(top_dist.keys()):
                vectors[topic_ind][tweet_ind][k]=top_dist[k]
    return vectors

In [78]:
def evaluate_vector(vector):
    as_top_num,tweet_num,lda_top_num=vector.shape[0],vector.shape[1],vector.shape[2]
    f=np.zeros(as_top_num)
    for i in range(as_top_num):
        ans=np.zeros(0)
        for j in range(tweet_num):
            top_dist=vector[i][j]
            import heapq
            kbig=np.array(heapq.nlargest(2,range(len(top_dist)), key=top_dist.__getitem__))
            #print(kbig)
            #print(ans)
            ans=np.concatenate([ans,kbig])
        from scipy import stats
        max_top=stats.mode(ans)[1][0]
        freq_max_top=np.sum(ans==max_top)
        f[i]=(max_top/(tweet_num*2))
    return f[0]

In [79]:
def evaluate(tfidf_corpus,tweets_dictionary,as_tweet,processed_doc,min_top_num,max_top_num):
    scores=np.zeros(max_top_num+1)
    for lda_top_num in range (min_top_num,max_top_num+1):
        lda_model=gensim.models.LdaMulticore(tfidf_corpus,num_topics=lda_top_num,id2word=tweets_dictionary,passes=10,random_state=20220916)
        vector=form_vector(lda_top_num,as_tweet,lda_model)
        try:
            scores[lda_top_num]=evaluate_vector(vector)
            print(lda_top_num,scores[lda_top_num])
        except:
            scores[lda_top_num]=-1
        
    return scores

In [80]:
scores=evaluate(tfidf_corpus,tweets_dictionary,ct,processed_doc,2,20)

2 0.5
3 0.45
4 0.4
5 0.35
6 0.3
7 0.3
8 0.35
9 0.25
10 0.4
11 0.25
12 0.35
13 0.2
14 0.2
15 0.25
16 0.4
17 0.45
18 0.4
19 0.2
20 0.45


In [87]:
lda_model=gensim.models.LdaMulticore(tfidf_corpus,num_topics=2,id2word=tweets_dictionary,passes=10,random_state=20220916)

In [88]:
pyLDAvis.enable_notebook()
LDAvis_prepared=pyLDAvis.gensim_models.prepare(lda_model,bow_corpus,tweets_dictionary)
LDAvis_prepared
pyLDAvis.save_html(LDAvis_prepared,'71013.html')

/root/miniconda3/envs/myconda/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/root/miniconda3/envs/myconda/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/root/miniconda3/envs/myconda/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/root/miniconda3/envs/myconda/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/root/miniconda3/envs/myconda/lib/python3.9/site-packages/past/b

In [89]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.002*"risk" + 0.002*"health" + 0.002*"get" + 0.001*"die" + 0.001*"know" + 0.001*"blood" + 0.001*"need" + 0.001*"death" + 0.001*"say" + 0.001*"infect"
Topic: 1 Word: 0.001*"think" + 0.001*"time" + 0.001*"need" + 0.001*"risk" + 0.001*"say" + 0.001*"tell" + 0.001*"like" + 0.001*"get" + 0.001*"health" + 0.001*"high"


In [90]:
def get_topic(text,lda):
    bow=tweets_dictionary.doc2bow(preprocess(text))
    tf=tfidf[bow]
    tmp=dict(lda_model[tf])
    ind=max(tmp,key=tmp.get)
    return ind

In [94]:
text["topic_ind"]=text.apply(lambda x:get_topic(x["cleaned_text"],lda_model),axis=1)

In [96]:
topic_path="/mnt/vaccine/old_dataset/topic_of_tweets/second_gen/"
topic_senti=np.zeros(2)
len_topic=np.zeros(2)
for i in range(2):
    tmp=text[text["topic_ind"]==i]
    topic_senti[i]=(tmp["senti"]).sum()
    len_topic[i]=len(tmp)
    tmp.to_csv(topic_path+str(i)+".csv")

In [95]:
text

cleaned_text   senti  topic_ind
0    true don't put students and parents at risk po... -0.5267          1
1    learning to live with covid and myself lifesty... -0.3612          1
2    no trial all trials over top secret data it wi... -0.3612          1
3    telling kevin about enrolling in the trial tod... -0.6604          0
4    i now desperately want to buy again those wale... -0.8957          1
..                                                 ...     ...        ...
395  if caught using a fake card to get into places... -0.9524          1
396  i see a few people outgassing about csl share ... -0.9246          1
397  look as a juris doctor i can tell you that liv... -0.9337          1
398  catholics the pope has said getting vaccinated... -0.8548          1
399  in vaccine science a reduced immune response i... -0.9759          0

[400 rows x 3 columns]